In [1]:
import numpy as np
import pandas as pd
import scipy as sp

from scipy.sparse import random, coo_matrix, lil_matrix, dok_matrix, csr_matrix, csc_matrix

# BLU10 - Learning Notebook - Part 1 of 3 - Introduction to Recommender Systems

# 0 Introduction

## 0.1 What are Recommender Systems? Are they really important?

>"35 percent of what consumers purchase on Amazon and 75 percent of what they watch on Netflix come from product recommendations based on such algorithms." [(link)](https://www.mckinsey.com/industries/retail/our-insights/how-retailers-can-keep-up-with-consumers)


**Recommender Systems** (RS) are widely used in companies providing a wide range of similar content (e.g. movies/shows on Netflix, songs/podcasts on Spotify, items on Amazon). Since these companies cannot ask all users to rate every single content (Spotify has [+50 million tracks and 232 million active users)](https://newsroom.spotify.com/company-info/), recommender systems enable the companies to suggest new content for both users which already showed their preferences and to new customers they don't know anything about.

---

*That seems nice, but why can't we use Classification or Regression Models we already know?*

The recommender system aims to predict the best ranking possible of items for a user. If we are trying to predict a rating, and a higher rating is a better score, why can't we consider this as a classical regression or an ordinal classification \*[1] problem? The main aspect of recommender systems (when compared to typical classification/regression problems) is that we are solving a high-sparsity problem on our inputs (meaning, there are a lot of zeros!). We normally have a severe imbalance for unlabeled data, so we consider this task as a matrix completion problem.


>*[1] Ordinal classification considers the label as a classification problem where the order has meaning (e.g. predicting the rating of a movie where an higher rating indicates a better movie). Typical classifications problems, such as predicting the weather labels (rain vs sunny vs cloudy) do not have an intrinsic order (e.g. "sunny is lesser than cloudy").*

## 0.2 Notorious Use Cases

**Youtube**: Google-powered, their video recommendation system uses content (e.g. metadata) and user activity (implicit and explicit) data and it is responsible for 60% of video clicks from the home page. For curious minds, here is a link for their [(paper)](https://www.researchgate.net/profile/Sujoy_Gupta2/publication/221140967_The_YouTube_video_recommendation_system/links/53e834410cf21cc29fdc35d2/The-YouTube-video-recommendation-system.pdf)

**Spotify**: the Swedish company uses a mix of Collaborative Filtering, NLP, Raw Audio models and Music Curators to suggest content for its users. Their recommender system is feeded not by explicit ratings but by users' interaction with the software (implicit feedback). 

**Netflix**: by mixing implicit and explicit feedback, the media-services provider uses an interface with top-down right-to-left ranking for suggested personalized content. They use your interactions with the service, similarity to other members' tastes and metadata about the titles (e.g. genre, actors) to feed their Recommendation System. Simple English explanation from Netflix themselves [(here)](https://help.netflix.com/en/node/100639).

## Types of Recommender systems
- <span style="color:darkred">Non-personalized systems</span>: Recommendations are the same for every user
    - **Best-Seller**
    - **Trending Hot**
    - **Highest rated**
    - **People who like X also like Y**
    
    
- <span style="color:darkred">Personalized systems</span>: Recommendations are specific for each user

    - **Collaborative Filtering**: Based only on the users past behavior 
        - **User-based**: Find similar users to me and recommend what they liked
        - **Item-based**: Find similar items to those that I have previously liked
    - **Content-based**: Recommend based on item features 
    - **Demographic**: Recommend based on user features
    - **Personalized Learning to Rank**: Treat recommendation as a ranking problem 




## 0.3 Agenda
The main goal of BLU10 is for you to learn how to build "Non-personalized recommender systems". After that, in BLU11 you will learn how to build "Personalized recommender systems" and then we will end with the "Workflow" in BLU12.

# 1 Framework

**Recommendation Systems (RS)** are software systems that recommend items to users, that they might like.

We start by learning the main components of an RS.

![Recommender Sytems Framework](./media/recommender_systems_framework.png)

*Fig.1 - RS framework with a community, plus the basic and extended models.*

We refer back to this framework frequently throughout the specialization, but for now, let's drill down into each component of the above figure:

## 1.1 Users

The *consumers* or people, denoted by $U = \{u_1, u_2, ..., u_m\}$, where the number of users $\left\vert{U}\right\vert = m$.

We reserve the indexing letters $u$ and $v$ to denote generic individual users.

## 1.2 Items

The *products* or things, a set $I = \{i_1, i_2, ..., i_n\}$, with the number of elements $\left\vert{I}\right\vert = n$. 

The indexing letters for items are $i$, $j$ and $l$.

## 1.3 Ratings

**Ratings** are the *transactions* or *opinions*, provided by the users about the items.

We write the set $R = \{r_{u_1, i_1}, ..., r_{u_m, i_n}\}$, where each rating $r_{u, i}$ corresponds to a user-item pair $(u, i) \in U \times I$.

* Any user $u \in U$ can make no more than one rating $r_{u, i}$ for a particular item $i \in I$.
* Any user $u \in U$ is free to rate, or not, any number of items $i \in I$, including none.

Non-personalized recommender systems usually make exclusive use of this matrix in order to make predictions. These will be the first ones we'll study and in the next section we will explore them more thoroughly due to their importance for recommender systems.



## 1.4 User model

As introduced above, RS are in the business of matching users and items.

Sometimes, it's convenient to have user and item profiles in the same attribute space $A$.

The user model $M$, defines $M = \{m_{u_1, a_1}, ..., m_{u_m, a_r}\}$, for $(u, a) \in U \times A$, where $A$ is the set of item attributes.

This matrix is normally used for collaborative filtering systems (or user-based filtering). These are the first of the personalized systems (BLU11) we will learn in subsequent notebooks.


## 1.5 Profiles

**Profiles** are a collection of objects (users or items, in our framework) and their **attributes**.

Consider the set of attributes $A = \{a_1, ..., a_r\}$, where $r \in \mathbb{N_1}$. Let be $A$ an arbitrary set of **item attributes**.

We can define profiles $P = \{p_{i_1, a_1}, ..., p_{i_n, a_r}\}$, where $(i, a) \in I \times A$ and values $p_{i, a}$ indicate the presence of $a$ in $i$.

Let $B$ be an arbitrary set of user characteristics, and we can apply the same reasoning to build **user profiles**.

Typically, item profiles contain information about the content of the items, and user profiles are more focused on demographics or other characteristics regarding the user.

This matrix is normally used for content-based filtering systems, the last ones we'll learn.

# 2 Ratings matrix

The community matrix hints at the canonical representation of the ratings matrix, at the core of any RS.

We represent the set $R = \{r_{u_1, i_1}, ..., r_{u_m, i_n}\}$ as a $U \times I$  matrix - the **ratings matrix** -, where the values are the ratings $r_{u, i}$, if they exist:


<img align="center" width="413" height="239" src="./media/ratings_matrix.png">


We represent not recorded ratings as zeros or missing values, enforcing the $U \times I$ shape.

We'll come back to the rating matrix with more detail in the next notebook!

# 3 Recommendations

The whole objective of Recommender Systems is to fill in the blanks in our ratings matrix and return the best possible items to the user. So bear this in mind, for each user we want to find, **for the items he hasn't ranked**, the ones that he is most likely to rank - the same rationale can be applied for a user that might want to buy something.

Throughout the course, we learn different ways to predict unseen ratings and we will start with the Non-Personalized approach that does not take into account specific information about the user.

## 3.1 Prediction step

The RS core computation is to predict the utility of unseen items $i \in I \setminus I_u$ to a user $u$, where $I \setminus I_u$  is the subset of items rated by user $u$.

At the core, we learn a function $f$ that maps user-item pairs into ratings $f : U \times I \to S$ given by $\hat{r}_{u, i} = f(u, i)$ where $S$ is the set of possible ratings.

Once we have it, there are two main types of recommendations: top-$N$ and best-item.

## 3.2 Top-*K* items

For a given user $u \in U$, we need a set of predictions $\hat{R}_u = \{f(u, i) : (u, i) \in u \times (I \setminus I_u)\}$.

Then, we take a subset $L_u \subset (I \setminus I_u$), containing the items with the *k*-largest predicted ratings $\hat{r}_{u, i} \in \hat{R}_u$.

Optionally, $L_u = \{i_0, i_1, ..., i_k\}$ can be ordered as $\hat{r}_{u, i_0} \geq \hat{r}_{u, i_1} \geq ... \geq \hat{r}_{u, i_k}$.

## 3.3 Best-item

Can be seen as a particular case of top-$K$, with $K = 1$.

Thus, the function $f(u, i)$ can be used to find the best item, such as $j = \underset{i \space \in \space I \setminus I_u}{\mathrm{argmax}} \space f(u, i)$.



As you see, there are many operations that can be applied to the ratings matrix in order to extract useful information for all the users, most importantly you should have this in the back of your mind...

<img align="center" width="413" height="239" src="media/meme_rmatrix.jpg">

We will speak about this structure and the ratings matrix in notebook number 2 of this BLU.

# 4 Context-awareness

Finally, some systems, particularly Personalized RS's, consider the context, alongside users and items. Take $C$ as a set of contexts.

The reasoning is that sometimes the utility for an item is observed to depend on other variables.

(A very good camera, for example, may be of lesser utility for a newbie than it is for a pro, for example.)

In these cases, we need $f$ to use the context as well, as $f : U \times I \times C \to S$ given by $\hat{r}_{u, i, c} = f(u, i, c)$.

For a given user $u \in U$, the predictions become $\hat{R}_u = \{f(u, i, c) : (u, i, c) \in u \times (I \setminus I_u) \times C\}$.

## 4.1 Time

We can think of time as a particular case of context-aware RS, where $\hat{r}_{u, i, t} = f(u, i, t)$.

As you might imagine, this rating matrix (for most RS) can get huge in almost no time. If you have 100 users and just 1000 products you get a million combinations of Users and Products with a high probability of having a real sparse matrix (very unlikely those 100 users recommended 1000 products). 
<br>
<br>
**Now think about Amazon customer and product base..**

**Next we are going to see how we can map users to products and construct a rating matrix from scratch! See you in a bit!**